# Base de datos Riak

In [1]:
import riak
from pprintpp import pprint as pp
import json
import pandas as pd

import uuid
import hashlib

## Conexion a RIAK

In [2]:
# connect to database
cliente = riak.RiakClient()
cliente.ping()

True

## Funciones utiles para el desarrollo de la práctica

In [3]:
# ELimina datos de un Bucket
def drop_keys(bucket):
    for keys in bucket.stream_keys():
        # print(keys)
        for keys in bucket.stream_keys():
            for key in keys:
                bucket.delete(key)

In [4]:
# Devuelve el hash de un string
def hash_string(s):
    s_utf8 = s.encode('utf-8')
    return hashlib.md5(s_utf8).hexdigest()

In [5]:
# Convertimos el objeto JSON en un objeto pandas 
def json_to_pandas(jsonObj):
    return pd.read_json(json.dumps(jsonObj))

## Lectura de la informacion en Pandas

In [6]:
df_dificultades = pd.read_csv("./data/dificultades.csv",encoding='utf-8');
df_encadenamientos = pd.read_csv("./data/tipos_encadenamiento.csv", encoding='utf-8');
df_escaladores = pd.read_csv("./data/escaladores_lite_2017.csv", encoding='utf-8');
df_ascensos = pd.read_csv("./data/ascensos_lite_2017.csv", encoding='utf-8');

In [7]:
# 'merge' del tipo de encadenamiento en el data frame de ascensos
df_ascensos = pd.merge(df_ascensos, df_encadenamientos, on = ['id_tipo_encadenamiento'], how = 'inner')
df_ascensos = df_ascensos[['id_escalador','id_dificultad', 'tipo_encadenamiento', 'nombre_via', 'risco', 'sector', 'pais']];


## Desnormalización del data frame de ascensos

Voy a incluir la información del usuario en cada registro de ascenso, para ir creando los buckets que van interesando según las preguntas a resolver


In [8]:
df_ascensos_desnormalizado = pd.merge(df_ascensos, df_escaladores, on = ['id_escalador'], how = 'inner');
df_ascensos_desnormalizado = pd.merge(df_ascensos_desnormalizado, df_dificultades, on = ['id_dificultad'], how = 'inner');

#renombrar campos autonombrados por merge
df_ascensos_desnormalizado = df_ascensos_desnormalizado.rename(
    columns={
        'pais_x' : 'pais_risco',
        'ciudad':'ciudad_escalador',
        'pais_y':'pais_escalador'});

df_ascensos_desnormalizado.head()

,id_escalador,id_dificultad,tipo_encadenamiento,nombre_via,risco,sector,pais_risco,nombre,sexo,fecha_nacimiento,ciudad_escalador,pais_escalador,comienzo,grado_frances,grado_usa,grado_bloque_usa
0,66274,25,Toprope,KVARTETTEN,LASSEHAGA,BôNN,SWE,Sara Herrmann,Hombre,2555-12-31,Lysekil,SWE,0,4c,5.6,V0
1,29844,25,Toprope,"FORGIVE ME AMY, FOR I HAVE SENT",RED RIVER GORGE,GOLD COAST,USA,Tom Burton,Hombre,1962-04-21,Ogden,USA,1992,4c,5.6,V0
2,56407,25,Toprope,EL TIó LILA,MONT-RAL,TERRANEGRA,ESP,Susana Barrera Vilarmau,Mujer,1978-01-19,Manresa,ESP,0,4c,5.6,V0
3,66273,25,Toprope,GUNNENG,KOLSåS,ØVRE SYDSTUP,NOR,Erling Fasting,Hombre,1988-01-08,Trondheim,NOR,2016,4c,5.6,V0
4,65841,25,Toprope,MYRZEBIAKI,DOLINA SZKLARKI,SLONECZNE SKALY,POL,Justyna Wosik,Mujer,2555-12-31,Sosnowiec,POL,0,4c,5.6,V0


## PREPARACIÓN DE LA BASE DE DATOS

Para resolver las preguntas me interesará tener los siguientes buckets:

|bucket|tipo|clave|Contenido del bucket|
|-|-|-|
|dificultades|default|id_dificultad|dificicultad en grado frances, usa y bloque-usa|
|ascensos|default|generada con identificador unico|Información desnormalizada de cada ascenso|
|ascensos_acum|'map'|id_escalador|Un mapa por escalador con: nombre, sexo, pais, ascensos_acumulados, suma_dificultades|

** Indices por bucket **

|Bucket|Nombre del índice|Contenido|¿Que busquedas permite hacer?|
|-|-|-|-|
|ascensos|idx_IdEscalador_int|Id del escalador|Localizar movimientos de un escalador concreto|



**CREACIÓN DE JSON CON LA INFORMACIÓN QUE IRÁ A LOS BUCKETS**

In [9]:
ascensosJsonString = df_ascensos_desnormalizado.to_json(orient = 'records')
ascensosJson = json.loads(ascensosJsonString)

In [10]:
dificultadesJsonString = df_dificultades.to_json(orient = 'records')
dificultadesJson = json.loads(dificultadesJsonString)

**CREACIÓN DE BUCKET ASCENSOS ACUMULADOS**

In [11]:
BUCKET_ACUM_ASCENSOS = 'acum_ascensos'
bucket_acum_ascensos = cliente.bucket_type('maps').bucket(BUCKET_ACUM_ASCENSOS)
drop_keys(bucket_acum_ascensos)

for ascensoJson in ascensosJson:
    # construcción de ascensos acumulados por persona
    acumulador_ascensos = bucket_acum_ascensos.new(str(ascensoJson['id_escalador']))
    acumulador_ascensos.registers['nombre'].assign(ascensoJson['nombre'].encode('utf-8'))
    acumulador_ascensos.registers['sexo'].assign(ascensoJson['sexo'].encode('utf-8'))
    acumulador_ascensos.registers['pais'].assign(ascensoJson['pais_escalador'].encode('utf-8'))
    acumulador_ascensos.counters['numero_vias'].increment()
    acumulador_ascensos.counters['suma_dificultades'].increment(ascensoJson['id_dificultad'])
    acumulador_ascensos.store()

**CREACIÓN DE BUCKET ASCENSOS**

In [12]:
BUCKET_ASCENSOS = 'ascensos'
bucket_ascensos = cliente.bucket(BUCKET_ASCENSOS)
drop_keys(bucket_ascensos)

for ascensoJson in ascensosJson:
    key = uuid.uuid1().hex
    ascenso = bucket_ascensos.new(key, ascensoJson)
    ascenso.add_index('idx_IdEscalador_int', ascensoJson["id_escalador"])
    ascenso.store()

**CREACIÓN DE BUCKET DIFICULTADES**

In [13]:
BUCKET_DIFICULTADES = 'dificultades'
bucket_dificultades = cliente.bucket(BUCKET_DIFICULTADES)
drop_keys(bucket_dificultades)

for dificultadJson in dificultadesJson:
    dificultad = bucket_dificultades.new(str(dificultadJson["id_dificultad"]), dificultadJson)
    dificultad.store()

## LECTURA DE LOS BUCKETS RIAK

**DATA FRAME ASCENSOS ACUMULADOS**

In [14]:
ascensosAcumuladosJson = []
for streamKeys in bucket_acum_ascensos.stream_keys():
    for streamKey in streamKeys:
        acumulador_ascensos = bucket_acum_ascensos.get(streamKey)
        ascensosAcumuladosJson.append({'id_escalador' : streamKey,
                                       'nombre' : acumulador_ascensos.registers['nombre'].value,
                                       'sexo' : acumulador_ascensos.registers['sexo'].value,
                                       'pais' : acumulador_ascensos.registers['pais'].value,
                                       'numero_vias' : acumulador_ascensos.counters['numero_vias'].value,
                                       'suma_dificultades' : acumulador_ascensos.counters['suma_dificultades'].value})
        
# Convertimos el objeto json en un objeto pandas 
df_bucket_ascensosAcumulados = json_to_pandas(ascensosAcumuladosJson)
df_bucket_ascensosAcumulados.head(3)

,id_escalador,nombre,numero_vias,pais,sexo,suma_dificultades
0,49416,Oliver Gonzalez Betancourt,1,DEU,Hombre,40
1,49229,Matthias Meilick,7,DEU,Hombre,316
2,49003,Enea Bionda,6,CHE,Hombre,267


**DATA FRAME DIFICULTADES**

In [15]:
filasDificultades = []
for keys in bucket_dificultades.stream_keys():
    for key in keys:
        filasDificultades.append(bucket_dificultades.get(key).data)
    
df_bucket_dificultades = json_to_pandas(filasDificultades);
df_bucket_dificultades.head(3)

,grado_bloque_usa,grado_frances,grado_usa,id_dificultad
0,VB,2b,5.1,9
1,V11,8a,5.13b,62
2,V5/V6,6c+,5.11b,46


## RESPUESTAS A LAS PREGUNTAS

### 1.a) Los 10 escaladores (hombres) más activos. (orden auxiliar por Id)

In [17]:
df = df_bucket_ascensosAcumulados[df_bucket_ascensosAcumulados['sexo']=='Hombre'];
df = df.sort_values(['numero_vias', 'id_escalador'],ascending=[False, False]);
df[['id_escalador', 'nombre', 'sexo', 'pais', 'numero_vias']].head(10)

,id_escalador,nombre,sexo,pais,numero_vias
2818,50884,Christopher Leonetti,Hombre,USA,43
2418,20095,Matthias Schuster,Hombre,DEU,33
4568,9171,Laurenz Trawnicek,Hombre,AUT,33
1285,20384,Clemens Kurth,Hombre,DEU,24
1277,23550,Moritz Perwitzschky,Hombre,DEU,22
2042,62639,Artur Gryt,Hombre,POL,21
548,30439,Nuno Henriques,Hombre,PRT,20
1337,37830,Rodrigo Pessoa,Hombre,BRA,19
2584,60616,Alex Garriga,Hombre,ESP,17
514,40521,Brandon Gezel,Hombre,USA,17


### 1.b) Los 10 escaladoras (mujeres) más activas. (orden auxiliar por Id)

In [18]:
df = df_bucket_ascensosAcumulados[df_bucket_ascensosAcumulados['sexo']=='Mujer'];
df = df.sort_values(['numero_vias', 'id_escalador'],ascending=[False, False]);
df[['id_escalador', 'nombre', 'sexo', 'pais', 'numero_vias']].head(10)

,id_escalador,nombre,sexo,pais,numero_vias
2806,53983,Daniela Bärtschi,Mujer,CHE,16
2089,54695,Elfi Hasler,Mujer,AUT,13
2857,65502,Ksenia Targosz,Mujer,POL,12
3950,51142,Isabelle Kölle,Mujer,AUT,12
2357,44050,Betka Galicic,Mujer,SVN,12
1338,37559,Magdalena Trzemzalska,Mujer,POL,11
4513,60268,Annemarie Van den Berg,Mujer,NLD,10
3809,55467,Katrin Gründler,Mujer,DEU,10
2118,49469,Line Tveter,Mujer,NOR,10
1766,22291,Stella Mascari,Mujer,USA,10


### 2. Lista de los 10 ascensos "On sight" de la escaladora más activa en orden decreciente de dificultad (y por nombre ascendete)

En este caso voy a obtener los **ascensos de la escaladora más activa** (de la pregunta anterior se que id_escalador=53983) **accediendo por el indice al bucket de ascensos**:

In [21]:
filasAscensos = []
streamIndex = bucket_ascensos.stream_index("idx_IdEscalador_int", 53983)
for keys in streamIndex.results:
    for key in keys:
        filasAscensos.append(bucket_ascensos.get(key).data)
    
df_ascensos = json_to_pandas(filasAscensos);
#df_ascensos.head(3)

In [22]:
df_ascensos = df_ascensos[df_ascensos['tipo_encadenamiento']=='Onsight']
df_ascensos = df_ascensos.sort_values(['id_dificultad', 'nombre_via'],ascending=[False, True])
df_ascensos[['nombre_via', 'grado_frances', 'risco', 'pais_risco']].head(10)


,nombre_via,grado_frances,risco,pais_risco
5,UNDA NIEDDA,6b+,GUTTURU CARDAXIUS,ITA
10,TIRO GHIRO,6a+,BRONTALLO,CHE
11,BRIDGE,6a,CALA FUILI,ITA
6,DOLCE EDERA,6a,ISELLE,CHE
13,MATèRIA FOSCA,6a,SIURANA,ESP
15,QUIDAM,5c,JOLIMONT,CHE
0,MATEDO,5b,SAN NICOLò,ITA
9,MOIJE.COM,5b,LE RIF D'ORIOL,FRA
1,BOLA DE DRAP,5a,ARBOLI,ESP


### 3. Dificultad media de los ascensos del escalador más activo

De la pregunta 1 ya tenemos un data frame con los datos acumulados por escalador. Utilizandolo podemos obtener la media de los ascensos del escalador más activo (id_escalador = 50884):

In [41]:
df=df_bucket_ascensosAcumulados[df_bucket_ascensosAcumulados['id_escalador']==50884]
id_dificultad_media = int(round(df['suma_dificultades']/df['numero_vias'],0))

df = df_bucket_dificultades[df_bucket_dificultades['id_dificultad']==id_dificultad_media];
df = df[['grado_frances']]
df.columns=['Dificultad media de ascensos escalador mas activo']
df

,Dificultad media de ascensos escalador mas activo
37,6


### 4.a) Los 10 ascensos mas dificiles

### 4.b) Los 10 ascensos mas dificiles a vista (On sight)

### 5.a) Grado medio y maximo de los ascensos en España de los 10 escaladores NO ESPAÑOLES con mas ascensos en España

### 5.b) Grado medio y maximo de los ascensos en España de los 10 escaladores ESPAÑOLES con mas ascensos en España

### 6.a) Dificultad media y maxima de los ascensos NO "Top Rope" de los escaladores con menos de 3 años de experiencia

### 6.b) Dificultad media y maxima de los ascensos NO "Top Rope" de los escaladores con entre 10 y 30 años de experiencia

### 7. Los 10 riscos españoles (o zonas) con mas ascensos por orden decreciente de numero de ascensos

### 8.a) Los 10 sectores españoles con mayor nivel de difcultad media de ascensos ordenadas por orden decreciente de dificultad y por numero de ascensos decreciente

### 8.b) Las 10 sectores españoles con menor nivel de dificultad medio de ascensos ordenadas por orden creciente de dificultad y por numero de ascensos decreciente